In [116]:
import os
import re
import yaml
import pandas as pd


In [222]:
df_results = pd.DataFrame(columns=['meta', 'hid_num', 'epochs', 'worker', 'time_started', 'time_completed', 'duration_seconds', 'path'])
params = {'meta': 'meta', 'epochs per task': 'epochs', 'hidden layers': 'hid_num'}
ix = 0
traverse = os.walk('.', topdown=False)
while True:
    try:
        dir = traverse.__next__()
        # find last-child dir
        if not dir[1] and dir[0] != "./.ipynb_checkpoints":
            # open hyperparams file and get meta, hid_num, epochs
            with open(dir[0] + '/hyperparameters.txt', 'r') as f:
                file = yaml.load(f, Loader=yaml.Loader)
                for param, df_param in params.items():
                    for item in file:
                        if param in item.keys():
                            if type(item[param]) == list:
                                df_results.loc[ix, df_param] = item[param][0]
                            else:
                                df_results.loc[ix, df_param] = item[param]
            
            # get worker number
            worker_num = re.findall(r"^./results (\d)", dir[0])[0]
            df_results.loc[ix, 'worker'] = worker_num
            
            time = dir[0].split('/')[-1].split('_')[0]
            date = dir[0].split('/')[-2]

            dt = (date + '-' + time)
            df_results.loc[ix, 'time_started'] = pd.to_datetime(dt, format='%Y-%m-%d-%H-%M-%S')
            
            df_results.loc[ix, 'old_path'] = dir[0]
            
            ix += 1
    except StopIteration:
        break

In [223]:
df_results['time_completed'] = df_results.groupby('worker').time_started.shift(-1)

df_results.dropna(inplace=True, subset=['time_completed'])
df_results.duration_seconds = pd.to_timedelta(df_results.time_completed - df_results.time_started).dt.total_seconds()

/Users/seba/miniforge3/envs/ml-base/lib/python3.9/site-packages/pandas/core/arrays/datetimelike.py:1187: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(


In [224]:
df_results['path'] = df_results.apply(lambda x: os.path.join('./results', str(x.meta)+"_"+str(x.hid_num)+"_"+str(x.epochs)), axis=1)

In [248]:
df_results.to_csv('results.csv', index=False)